## This Notebook is expected to be run on Google Colab

In [ ]:
!pip -q install langchain-huggingface pyarrow==15.0.2 langchain_community chromadb faiss-cpu ragas


## Mounting Google Drive

The FAQs data and the test data is stored in Google Drive under *mrp-project* folder ,So we need to mount that folder to the Notebook before we proceed.

In [ ]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'mrp-project')
drive.mount(MOUNTPOINT,force_remount=True)

Mounted at /content/gdrive


## Hugging Face API Authentication

Please make sure you stored Hugging Face API token in Google Colab Secrets in the *hf_token* key before you proceed

- https://huggingface.co/docs/hub/en/security-tokens
- https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75

In [ ]:

from huggingface_hub import login
from google.colab import userdata
def authenticate():
  hf_token=userdata.get('hf_token')
  login(hf_token)
  return hf_token

In [ ]:
hf_token=authenticate()

## Loading LLM

In [4]:
from langchain_huggingface import HuggingFaceEndpoint

def load_llm(model_name):
  llm = HuggingFaceEndpoint(
      repo_id=model_name,
      # model=model_name,
      task="text-generation",
      max_new_tokens=512,
      temperature=0.001,
      do_sample=False,
      huggingfacehub_api_token=hf_token


  )
  return llm


## Load Embedding Model

In [ ]:
## Loading Embedding Model
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
def load_embeddings(model_name):

  hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= model_name,
    task="feature-extraction",
  )

  return hf_embeddings

## Load Documents

In [ ]:
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
def load_data(file_path):
    df = pd.read_csv(file_path)
    loader = DataFrameLoader(df, page_content_column="document")
    return loader.load()

def split_texts(texts, chunk_size=1024, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(texts)



# Load Retriever

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

In [ ]:
from langchain.vectorstores import Chroma



def load_retriever(texts,embeddings,collection_name):
    vector_store = Chroma.from_documents(collection_name=collection_name,documents=texts, embedding=embeddings, persist_directory=collection_name)
    return vector_store,vector_store.as_retriever(search_kwargs={"k": 3,"score_threshold":.4}, search_type="similarity_score_threshold")





## Load Prompt

In [ ]:
from langchain.prompts.prompt import PromptTemplate

template = """You are an intelligent assistant helping Toronto Metropolitan University Website visitors on their frequently asked questions in English.

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
- if the answer to the question cannot be determined from the context alone or if the context is empty,only say "I cannot determine the answer to that"
- Use numbered lists whenever possible.
Context:
{context}
Question: {question}
Helpful Answer:"""

def load_prompt(template):
    return PromptTemplate(input_variables=["context", "question"], template=template)

# Extract Answer

In [ ]:
def get_text_after_last_helpful_answer(text):
    # Split the text by 'Helpful Answer:'
    parts = text.split('Helpful Answer:')
    # Get the last part
    last_part = parts[-1]
    return last_part.strip()

## RAG Pipeline(Testing with Llama-2 model and Embedding model intfloat)

In [ ]:
#Path to FAQs data in the Google Drive
data_file_path="gdrive/MyDrive/mrp-project/combined_faqs.csv"
llm_model_name='meta-llama/Llama-2-13b-chat-hf'
embedding_model='intfloat/e5-small-v2'
model_name=llm_model_name.split("/")[-1]
embedding_name=embedding_model.split("/")[-1]


In [ ]:
# Loading LLM
llm=load_llm(llm_model_name)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Loading Embedding Model
embeddings=load_embeddings(embedding_model)

In [ ]:
#Loading data
docs=load_data(data_file_path)
#Chunking documents
texts=split_texts(docs)

In [ ]:
#loading retriver

vector_store,retriever=load_retriever(texts,embeddings,embedding_name)

In [ ]:
# Testing Retriever

In [ ]:
results = vector_store.similarity_search("What are the admission requirements for Msc Data Science and Analytics?", k=2)
print(results)

[Document(metadata={'Department': 'Data Science and Analytics', 'Sub_Topic': 'Data Science and Analytics', 'Topic': 'Data Science and Analytics', 'question_answer': 'Minimum GPA of 3.00/4.33 (B) in the last two years of undergraduate studies\r\nStatement of Interest\r\n2 academic references\r\nResume/CV\r\nEvidence of completion of three prerequisite courses or equivalents', 'question_name': 'What are the admission requirements for data science and analytics program?', 'question_url': 'https://www.torontomu.ca/graduate/datascience/admission/faq/#accordion-content-1499954858059-what-are-the-admission-requirements-'}, page_content='What are the admission requirements for data science and analytics program?\nMinimum GPA of 3.00/4.33 (B) in the last two years of undergraduate studies\r\nStatement of Interest\r\n2 academic references\r\nResume/CV\r\nEvidence of completion of three prerequisite courses or equivalents'), Document(metadata={'Department': 'Ted Rogers School of Management', 'Sub

In [ ]:
docs = retriever.invoke("When can I change my course schedules for creative industries programs and how do I view my class schedule for these programs?")
print(docs)

[Document(metadata={'Department': 'Creative Industries', 'Sub_Topic': 'Class Schedules', 'Topic': 'Current Students', 'question_answer': 'You may view your schedule on MyServiceHub by selecting "Manage Classes", and then "View My Classes" or "My Weekly Schedule".\n\nMake sure to click the Next Week >> button to view your full weekly schedule. .You can adjust your course schedules during your course enrollment period.', 'question_name': 'How do I access my class schedule for creative industries programs?', 'question_url': 'https://www.torontomu.ca/creativeindustries/academic-advising/faqs/#accordion-content-1703033103003-how-do-i-access-my-class-schedule-'}, page_content='How do I access my class schedule for creative industries programs?\nYou may view your schedule on MyServiceHub by selecting "Manage Classes", and then "View My Classes" or "My Weekly Schedule".\n\nMake sure to click the Next Week >> button to view your full weekly schedule. .You can adjust your course schedules during

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain

#This function will create the Conversational Chain with Memory
def return_qa_chain (llm,retriever,prompt):
  # Loading prompt
  prompt=load_prompt(template)

  #Initializing memory
  memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_messages=True,
    output_key='answer'
  )

  qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    return_generated_question = True,
    verbose=False,
    chain_type="stuff",
    combine_docs_chain_kwargs={'prompt': prompt},
  )
  return qa


In [ ]:
qa=return_qa_chain (llm,retriever,template)
result=qa({"question" : "What are the admission requirements for msc data science and analytics program?"})

In [ ]:
print(result)

{'question': 'What are the admission requirements for msc data science and analytics program?', 'chat_history': [], 'answer': '\nTo be eligible for admission to the MSc Data Science and Analytics program, you must meet the following requirements:\n\n1. Minimum GPA of 3.00/4.33 (B) in the last two years of undergraduate studies.\n2. Statement of Interest.\n3. 2 academic references.\n4. Resume/CV.\n5. Evidence of completion of three prerequisite courses or equivalents.\n\nNote: The three prerequisite courses cannot be counted towards the GPA admission requirement.', 'source_documents': [Document(metadata={'Department': 'Data Science and Analytics', 'Sub_Topic': 'Data Science and Analytics', 'Topic': 'Data Science and Analytics', 'question_answer': 'Minimum GPA of 3.00/4.33 (B) in the last two years of undergraduate studies\r\nStatement of Interest\r\n2 academic references\r\nResume/CV\r\nEvidence of completion of three prerequisite courses or equivalents', 'question_name': 'What are the 

In [ ]:
print(get_text_after_last_helpful_answer(result['answer']))

To be eligible for admission to the MSc Data Science and Analytics program, you must meet the following requirements:

1. Minimum GPA of 3.00/4.33 (B) in the last two years of undergraduate studies.
2. Statement of Interest.
3. 2 academic references.
4. Resume/CV.
5. Evidence of completion of three prerequisite courses or equivalents.

Note: The three prerequisite courses cannot be counted towards the GPA admission requirement.


In [ ]:
result["source_documents"][0].metadata['question_url']

'https://www.torontomu.ca/graduate/datascience/admission/faq/#accordion-content-1499954858059-what-are-the-admission-requirements-'

In [ ]:
#Function to pass the question to RAG chain and get the response back
def ask(question,qa_chain):

    result = qa_chain({"question": question})

    output=get_text_after_last_helpful_answer(result['answer'])

    return output

## Preparing Evaluation Dataset

Please make sure you stored OPEN  API token in Google Colab Secrets in the *open_api_key* key before you proceed

- https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key
- https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75


In [ ]:
import os
#getting open API key from Google Colab Secrets and  storing it to the environment variable OPENAI_API_KEY
os.environ['OPENAI_API_KEY'] =userdata.get('open_api_key')

import nest_asyncio
import os
import openai
from dotenv import load_dotenv
from datasets import Dataset
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness

)
from ragas import evaluate

# Access the API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAI API client
openai.api_key = api_key

nest_asyncio.apply()

In [ ]:
#This function will fetch the contexts from the retriever and answers from the RAG chain based on the questions from the evaluation/test dataset.
def generate_evaluation_data_lists(eval_path,retriever,qa):

  eval_df=pd.read_csv(eval_path)
  #Only first 10 rows are filtered for evaluation,from total of 50 rows.
  eval_df=eval_df.head(10)
  question=[]
  ground_truth=[]
  answer=[]
  contexts=[]
  for index,row in eval_df.iterrows():
      output=ask(row['questions'],qa)
      question.append(row['questions'])
      ground_truth.append(row['ground_truths'])
      answer.append(output)
      contexts.append([docs.page_content for docs in retriever.get_relevant_documents(row['questions'])])

  return question,ground_truth,answer,contexts

In [ ]:
question,ground_truth,answer,contexts=generate_evaluation_data_lists("gdrive/MyDrive/mrp-project/eval_data.csv",retriever)

In [ ]:
ground_truth

['The MRP is a project rather than a thesis. It is applied, whereas thesis is research-focused.',
 "Hot food that emits odours should not be consumed in the Academic Computing labs. Only cold, 'finger' food and bagged lunches are allowed. Users must dispose of waste and recycling in the appropriate bins and adhere to the policies, or they may be asked to leave.",
 "To add an existing Google Keep note to a Google Doc file using the 'Copy to Google Doc' feature, you can select the note in Google Keep, then click on the More menu and choose the 'Copy to Google Doc' option. This will insert the content of the note into the Google Doc.",
 'You can identify a preceptor with graduate education for your practicum placement by connecting with your assigned Course Professor. Once you have identified a potential preceptor and received consent from the Course Professor, you can make contact with the preceptor to discuss learning opportunities. Following this process, you can fill out a Practicum R

In [ ]:
question

['I just want to know the difference between MRP and thesis in the data science and analytics program?',
 'What are the restrictions on bringing food and drink into the Academic Computing labs?',
 'How can you add an existing Google Keep note to a Google Doc file using the "Copy to Google Doc" feature?',
 'How can I identify a preceptor with graduate education for my practicum placement?',
 'How can I reply to or forward just a single message in a conversation using the drop-menu?',
 'How do I change modules during Course Intentions or Enrolment periods and what info should I provide to the university?',
 'What does the referee get from AwardSpring for their recommendation letter?',
 'When can I change my course schedules for creative industries programs and how do I view my class schedule for these programs?',
 'What factors should influence decisions on e-mail protocol and retention for student communication, given FOI requests and email uncertainty?',
 'Can I apply for winter (Janua

In [ ]:
contexts

[['What is the difference between an MRP and a thesis  for data science and analytics program?\nThe MRP is a project rather than a thesis. It is applied, whereas thesis is research-focused.',
  'What is the difference between a major research paper and a thesis for philosophy?\nA Major Research Paper (MRP) is usually around 30-40 pages long, roughly modelled on a typical article in a philosophy journal. A Thesis is usually around 120 pages long, roughly modelled on a typical small monograph or book. Both are meant to be original contributions to a current philosophical debate. An oral defence is required for a thesis but not for an MRP.\nIf you decide to do an MRP, then you have to take an elective each term in your second year. If you decide to do a Thesis, then that is all you have to do in your second year. Both options have positives and negatives. This is an important decision, and we can help you make the decision that is right for you.',
  'How is the program offered  for data s

In [ ]:
answer

['MRP is a project, whereas thesis is research-focused.\nMRP is a one-year full-time or a two-year part-time option, whereas thesis is a two-year full-time option.\nMRP is usually around 30-40 pages long, whereas thesis is usually around 120 pages long.\nMRP does not require an oral defense, whereas thesis does.\nMRP allows for more flexibility in terms of course selection, whereas thesis is more focused on research.\nMRP is a good option for those who want to gain practical experience, whereas thesis is a good option for those who want to delve deeper into a specific research topic.',
 "1. Cold, 'finger' food and bagged lunches are allowed in the KHW71, KHW71F, KHW71A, KHW71B, KHW71C, KHW377, KHW379 labs.\n2. Hot food that emits odors is not allowed in the labs.\n3. Respect others by disposing of waste and recycling in the appropriate bins.\n4. Users may be asked to leave if they do not adhere to these policies.",
 'To add an existing Google Keep note to a Google Doc file using the "C

In [ ]:

#This function will take question,answer,ground_truth,contexta and use RAGAS library to generate scores

def evaluate_df(question,answer,ground_truth,contexts):
  # To dict
  data = {
      "question": question,
      "answer": answer,
      "contexts": contexts,
      "ground_truth": ground_truth
  }

  # Convert dict to dataset
  dataset = Dataset.from_dict(data)

  result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_similarity,
        answer_correctness
    ],
     raise_exceptions=False
  )

  df = result.to_pandas()

  return df

In [ ]:
#This function will save the evaluation results of each LLM and Embedding Model as CSV file
def post_eval(df,model_name,embedding_name):
  df['llm'] = model_name
  df['embedding_model'] = embedding_name
  df.to_csv(f'eval_results_{model_name}_{embedding_name}.csv', index=False)
  return df


In [ ]:
df=evaluate_df(question,answer,ground_truth,contexts)
df=post_eval(df,model_name,embedding_name)
df.head(10)





Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall,answer_similarity,answer_correctness,llm,embedding_model
0,I just want to know the difference between MRP...,"MRP is a project, whereas thesis is research-f...",[What is the difference between an MRP and a t...,The MRP is a project rather than a thesis. It ...,1.000000,0.923077,0.970482,1.0,0.902579,0.663145,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
1,What are the restrictions on bringing food and...,"1. Cold, 'finger' food and bagged lunches are ...",[Are food and drink allowed in the Academic Co...,Hot food that emits odours should not be consu...,1.000000,1.000000,0.966125,1.0,0.919495,0.576028,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
2,How can you add an existing Google Keep note t...,To add an existing Google Keep note to a Googl...,[How do I integrate Google Keep with Google Dr...,To add an existing Google Keep note to a Googl...,0.833333,0.166667,0.977039,0.5,0.974075,0.743519,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
3,How can I identify a preceptor with graduate e...,You can identify a preceptor with graduate edu...,[Can I find my own preceptor?\nYes. You should...,You can identify a preceptor with graduate edu...,1.000000,0.000000,0.960642,1.0,0.930295,0.232574,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
4,How can I reply to or forward just a single me...,Please provide the follow-up question.,[Replying / Forwarding A Single Message\nYou c...,To reply to or forward just a single message i...,1.000000,1.000000,0.934314,1.0,0.751182,0.437796,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
5,How do I change modules during Course Intentio...,To change modules during Course Intentions or ...,[Would I be able to change my modules?\nYou ar...,To change modules during Course Intentions or ...,1.000000,1.000000,0.872946,1.0,0.974845,0.743711,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
6,What does the referee get from AwardSpring for...,There is no direct reward or benefit that a re...,[How does my referee submit their letter of re...,Your referee will receive an email from AwardS...,1.000000,1.000000,0.880061,1.0,0.853373,0.213343,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
7,When can I change my course schedules for crea...,You can change your course schedules for creat...,[How do I access my class schedule for creativ...,You can adjust your course schedules during yo...,0.833333,1.000000,0.907393,1.0,0.897100,0.411775,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
8,What factors should influence decisions on e-m...,1. Be careful about the content of emails as t...,[Q11: What should be my e-mail protocol and re...,Factors that should influence decisions on e-m...,1.000000,1.000000,0.909282,1.0,0.924811,0.699953,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2
9,Can I apply for winter (January) admission to ...,"1. Unfortunately, TMU does not offer winter ad...",[Can I apply for winter (January) admission fo...,TMU does not offer winter admission to the Arc...,1.000000,0.750000,0.000000,1.0,0.911623,0.477906,meta-llama/Llama-2-13b-chat-hf,intfloat/e5-small-v2


## Evaluate LLMs and Embedding Models

In [ ]:

#This is the main function that takes a list of LLMs and Embedding models to evaluate along with FAQs data and Test dataset.

def evaluate_llm_embedding(llm_list,embedding_list,data_file_path,eval_file_path):

  print("Loading documents..........")
  #Loading data
  docs=load_data(data_file_path)
  print("Chunking documents..........")
  #Chunking documents
  texts=split_texts(docs)

  for llm in llm_list:
    for embedding in embedding_list:
      model_name=llm.split("/")[-1]
      embedding_name=embedding.split("/")[-1]
      print(f"Evaluating.......Model: {model_name}, Embedding: {embedding_name}")
      print("1. Loading LLM......")
      # Loading LLM
      loaded_llm=load_llm(llm)
      print("2. Loading Embedding Model......")
      # Loading Embedding Model
      loaded_embedding_model=load_embeddings(embedding)
      print("3. Loading Retriever......")
      vector_store,loaded_retriever=load_retriever(texts,loaded_embedding_model,embedding_name)
      print("4. Loading QA Chain......")
      qa=return_qa_chain (loaded_llm,loaded_retriever,template)
      print("5. Generating Evalutation Data Lists......")
      question,ground_truth,answer,contexts=generate_evaluation_data_lists(eval_file_path,loaded_retriever,qa)
      print("6. Generating Evaluation Results......")
      df=evaluate_df(question,answer,ground_truth,contexts)
      print("7. Post processing and saving evaluation results......")
      df=post_eval(df,model_name,embedding_name)

  print("Comparing LLMs and Embeddings Finished....")




In [ ]:
#Hugging Face LLM model names
llm_list=['meta-llama/Llama-2-13b-chat-hf','meta-llama/Meta-Llama-3-70B-Instruct','mistralai/Mixtral-8x7B-Instruct-v0.1','NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO']
#Hugging Face Embedding model Names
embedding_list=['sentence-transformers/all-mpnet-base-v2','intfloat/e5-base-v2','BAAI/bge-small-en-v1.5','Snowflake/snowflake-arctic-embed-s']
#Path to the FAQs dataset
data_file_path="gdrive/MyDrive/mrp-project/combined_faqs.csv"
#Path to test dataset
eval_file_path="gdrive/MyDrive/mrp-project/eval_data.csv"
#Calling the main function to start 4*4 experiments
evaluate_llm_embedding(llm_list,embedding_list,data_file_path,eval_file_path)

Loading documents..........
Chunking documents..........
Evaluating.......Model: Llama-2-13b-chat-hf, Embedding: all-mpnet-base-v2
1. Loading LLM......
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
2. Loading Embedding Model......
3. Loading Retriever......
4. Loading QA Chain......
5. Generating Evalutation Data Lists......
6. Generating Evaluation Results......


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

7. Post processing and saving evaluation results......
Evaluating.......Model: Llama-2-13b-chat-hf, Embedding: e5-base-v2
1. Loading LLM......
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
2. Loading Embedding Model......
3. Loading Retriever......
4. Loading QA Chain......
5. Generating Evalutation Data Lists......
6. Generating Evaluation Results......


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

7. Post processing and saving evaluation results......
Evaluating.......Model: Llama-2-13b-chat-hf, Embedding: bge-small-en-v1.5
1. Loading LLM......
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
2. Loading Embedding Model......
3. Loading Retriever......
4. Loading QA Chain......
5. Generating Evalutation Data Lists......
6. Generating Evaluation Results......


...

7. Post processing and saving evaluation results......
Evaluating.......Model: Nous-Hermes-2-Mixtral-8x7B-DPO, Embedding: bge-small-en-v1.5
1. Loading LLM......
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
2. Loading Embedding Model......
3. Loading Retriever......
4. Loading QA Chain......
5. Generating Evalutation Data Lists......
6. Generating Evaluation Results......


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

7. Post processing and saving evaluation results......
Evaluating.......Model: Nous-Hermes-2-Mixtral-8x7B-DPO, Embedding: snowflake-arctic-embed-s
1. Loading LLM......
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
2. Loading Embedding Model......
3. Loading Retriever......
4. Loading QA Chain......
5. Generating Evalutation Data Lists......
6. Generating Evaluation Results......


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

7. Post processing and saving evaluation results......
Comparing LLMs and Embeddings Finished....
